# 2025PL作業六
# 🧪   Lab: 探索 Python 記憶體管理系統


### ✅ 作業目標：
- 學會如何觀察與分析 Python 的記憶體分配與釋放行為  
- 熟悉 `sys.getrefcount()` 與 `gc` 模組的使用  
- 學會避免記憶體洩漏與處理循環參照  



### 📤 繳交說明：
- 請填寫所有空白程式區塊和結果說明
- 完成後上傳網路學園 `學號姓名PL2025作業六.ipynb` 檔案


In [1]:
import sys
import gc
import weakref
import psutil
import os
import time

## 2️⃣ 物件記憶體大小與引用計數

In [ ]:
a = []
print("記憶體大小 (bytes):", sys.getsizeof(a))
print("引用計數:", sys.getrefcount(a))


### 🧩 任務 2.1
建立一個自訂的類別 `MyData`，實例化後觀察以下資訊：
- 實體的記憶體大小（使用 `sys.getsizeof()`）
- 引用計數（使用 `sys.getrefcount()`）


In [2]:
# ✅ 請在此撰寫你的程式碼
class Mydata():
    def __init__(self):
        self.data = []
obj = Mydata()
print("\nMyData 實體記憶體大小 (bytes):", sys.getsizeof(obj))
print("MyData 實體引用計數:", sys.getrefcount(obj))





MyData 實體記憶體大小 (bytes): 48
MyData 實體引用計數: 2


結果說明:


## 3️⃣ 垃圾回收與 __del__ 方法

In [ ]:
class MyClass:
    def __del__(self):
        print("MyClass 實例已被回收")

obj = MyClass()
print("建立完畢")
del obj
print("已刪除 obj")


### 🧩 任務 3.1
建立兩個 `Node` 物件 `a` 和 `b`，彼此互相參考形成循環，然後刪除它們，使用 `gc.collect()` 強制回收並觀察結果。


In [3]:
# ✅ 請在此建立循環參照並觀察垃圾回收情形
class Node:
    def __init__(self):
        self.ref = None
        self.data = []
    def __del__(self):
        print("Node 已被回收")

# 關閉自動垃圾回收（觀察手動行為）
gc.disable()

# 建立互相參考的兩個物件 a, b
a = Node()
b = Node()
a.ref = b
b.ref = a

# 觀察目前還沒回收
print("建立後，未回收垃圾物件數量：", len(gc.garbage))

# 刪除變數
del a
del b

# 嘗試手動強制垃圾回收
print("執行 gc.collect() 前：")
unreachable = gc.collect()
print("不可達物件數量：", unreachable)

# 顯示垃圾清單中仍未被回收的物件
print("gc.garbage:", gc.garbage)

# 開啟自動垃圾回收
gc.enable()






建立後，未回收垃圾物件數量： 0
執行 gc.collect() 前：
Node 已被回收
Node 已被回收
不可達物件數量： 140
gc.garbage: []


結果說明:


## 4️⃣ 使用 gc 模組觀察與控制垃圾回收

In [ ]:
gc.set_debug(gc.DEBUG_LEAK)

class Cycle:
    def __init__(self):
        self.cycle = self

c = Cycle()
del c
unreachable = gc.collect()
print(f"不可達物件數量：{unreachable}")
print("垃圾回收統計資訊：")
print(gc.garbage)


### 🧩 任務 4.1
1. 關閉自動垃圾回收 (`gc.disable()`)
2. 建立大量會造成循環參照的物件
3. 觀察手動與非手動回收下的情況差異


In [2]:
# ✅ 請在此完成任務 4.1
gc.disable()
class Cycle:
    def __init__(self):
        self.cycle = self
objects = []
for _ in range(10000):
    obj = Cycle()
    objects.append(obj)
print("build 10000 object")
# 4. 刪除對這些物件的引用
del objects
print("delete")
print("尚未手動執行 gc.collect()，觀察垃圾回收情況...")
print("不可達物件數量 (collect 前)：", gc.collect(0))  # 僅統計，不執行回收
print("gc.garbage 長度：", len(gc.garbage))

# 5. 手動回收
print("開始手動垃圾回收...")
unreachable = gc.collect()
print("gc.collect() 回收物件數：", unreachable)
print("gc.garbage 長度：", len(gc.garbage))


build 10000 object
delete
尚未手動執行 gc.collect()，觀察垃圾回收情況...
不可達物件數量 (collect 前)： 19998
gc.garbage 長度： 0
開始手動垃圾回收...
gc.collect() 回收物件數： 64
gc.garbage 長度： 0


## 5️⃣ 觀察記憶體使用量（使用 psutil）

In [ ]:
process = psutil.Process(os.getpid())

def memory():
    return process.memory_info().rss / 1024 ** 2  # MB

print("當前記憶體使用量: ", memory(), "MB")


### 🧩 任務 5.1
撰寫程式大量配置記憶體（例如建立上百萬項的 list 或 dict），觀察記憶體變化。


In [2]:
# ✅ 請在此撰寫程式並觀察記憶體使用變化
process = psutil.Process(os.getpid())

def memory():
    return process.memory_info().rss / 1024 ** 2  # MB

print("當前記憶體使用量: ", memory(), "MB")
print("配置大量記憶體")
array = []
for _ in range(1,50000001):
    array.append(_)
print("配置完成")
print("當前記憶體使用量: ", memory(), "MB")



當前記憶體使用量:  71.0546875 MB
配置大量記憶體
配置完成
當前記憶體使用量:  2002.21484375 MB


## 📌 延伸挑戰（optional）僅供參考


- 使用 `tracemalloc` 模組追蹤記憶體配置來源
- 比較 `list`, `tuple`, `set`, `dict` 等資料結構的記憶體使用差異


In [3]:
import tracemalloc

def test_structure_memory(structure_type):
    if structure_type == 'list':
        return [i for i in range(1000000)]
    elif structure_type == 'tuple':
        return tuple(i for i in range(1000000))
    elif structure_type == 'set':
        return {i for i in range(1000000)}
    elif structure_type == 'dict':
        return {i: i for i in range(1000000)}
    return None


# 開始記憶體追蹤
tracemalloc.start()

structures = {}
for stype in ['list', 'tuple', 'set', 'dict']:
    snapshot1 = tracemalloc.take_snapshot()
    structures[stype] = test_structure_memory(stype)
    snapshot2 = tracemalloc.take_snapshot()

    # 計算記憶體差異
    stats = snapshot2.compare_to(snapshot1, 'lineno')
    top = stats[0]
    print(f" [{stype}] 使用記憶體: {top.size_diff / (1024*1024):.4f} MB")
    print(f" 來源: {top.traceback.format()[-1]}")

tracemalloc.stop()


 [list] 使用記憶體: 34.7533 MB
 來源:     return [i for i in range(1000000)]
 [tuple] 使用記憶體: 34.3255 MB
 來源:     return tuple(i for i in range(1000000))
 [set] 使用記憶體: 58.6966 MB
 來源:     return {i for i in range(1000000)}
 [dict] 使用記憶體: 66.6965 MB
 來源:     return {i: i for i in range(1000000)}


### 結果說明
記憶體使用量:
$$tuple < list < set < dict$$
| 結構類型    | 儲存 10,000 筆資料時記憶體大小 | 特性說明                   |
| ------- | ------------------- | ---------------------- |
| `tuple` | 最省空間                | 不可變、固定長度、結構簡單          |
| `list`  | 中等                  | 可變長度、儲存參照              |
| `set`   | 略高                  | 使用雜湊、避免重複              |
| `dict`  | 最高                  | 儲存 key-value、額外雜湊與映射成本 |
